# Dino Game model fitting

## Summary
* Initial go at dino game
* Read-in Spencer's data and do some very minimal pre-processing to make things play nicely with popular ML pacakges like sklearn, keras, and PyTorch
* Fit basic Softmax Regression model -- performs WAY better than I expected
* Though this nice performance could be super misguiding -- could be a result of the data structure I mentioend above, or the fact that these samples aren't IID (one frame of the game relies on another) so we are inherently violating the assumptions of a LogReg model. For this, I suggest a "thinning" type of approach across even more dino-games if we want to learn an actual mapping
* Will likely need a lot more data to train a deep-net, especially since LogReg performs so well.

### Inputs

* Spencer went all-out and sent massive image files in RGB channels (lol) so for now, I will just flatten these, drop that third channel (will mess up scaling of values in matrix though), and train on these vectors.

### Outputs

* A label prediction for the key to press (or what action the model should take given the pixel values in the image?)

### Modeling task
* Given an input image $X$, output a label for the action to be taken by the model (jump, duck, nothing)

### Evaluation metric
* Classification accuracy 
* Cross-entropy loss for training

### Models
* Multinomial logistic/Softmax regression
* ConvNets (1D and 2D) -- Spencer suggest ResNet, but I have so little knowledge in this field, I kinda of want to do a "survey" first
* SVM models?

### To-do
* Data pre-processing to make this task more "learnable" is needed. 
    * Images should be converted to grayscale -- models not learning well
    * Perhaps move from predicting the integer value of the key-press and move to a one-hot encoding (I can write a utility function to go back and forth from the two to make the actual dinosaur move)
   
* More modeling
     * Move to Google Collab for ConvNets -- LogReg was painfully slow on my local machine to fit
     * Flat try fancy models next!

In [13]:
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from pathlib import Path
from keras.utils.np_utils import to_categorical
from typing import List
import cv2 as cv
# Using SMOTE for the over sampling portion.
from imblearn.over_sampling import SMOTE
%matplotlib inline

In [14]:
target_address = os.path.join(Path(os.getcwd()).parent,'Window_capture\\Data\\command_keys.npy')
# screenshot_address = os.path.join(Path(os.getcwd()).parent,'Window_capture\\Data\\screenshots.npy')
screenshot_address = os.path.join(Path(os.getcwd()).parent,'Window_capture\\Data\\screenshots.npy')

labels = np.load(target_address)
images = np.load(screenshot_address, allow_pickle = True)


print("Length Command Keys Shape: ",labels.shape)
print("Length Screenshot Shape: ",images.shape)
print("Screenshot Shape: ",images[0].shape)

Length Command Keys Shape:  (15673,)
Length Screenshot Shape:  (15673, 129600)
Screenshot Shape:  (129600,)


In [15]:
np.unique(labels, return_counts = True) # We see quite a bit of imbalance among the do nothing / jump / duck

(array([-1, 38, 40]), array([12109,  1523,  2041], dtype=int64))

In [16]:
np.unique(labels == -1, return_counts = True)

(array([False,  True]), array([ 3564, 12109], dtype=int64))

In [17]:
# res_list = [i for i, value in enumerate(labels) if value == -1] # Let's get rid of some -1 values.
# idx = np.random.choice(res_list, 2000, replace=False) # Randomly choose X number of entries to be deleted specified as -1
# images = pd.DataFrame(images) # flatten images then converted to dataframe for easier removal of idx
# images = np.array(images.drop(images.index[idx])) # flatten images then converted to dataframe for easier removal of idx
# labels = np.delete(labels, idx)
# print(images.shape, labels.shape)
# print(np.unique(labels, return_counts = True))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.25)

In [19]:
np.unique(y_train, return_counts = True)

(array([-1, 38, 40]), array([9058, 1162, 1534], dtype=int64))

In [20]:
#Oversampling the data
smote = SMOTE(random_state = 101)
X_train_samp, y_train_samp = smote.fit_resample(X_train, y_train)
np.unique(y_train_samp, return_counts = True) # Oversampled Balanced.

(array([-1, 38, 40]), array([9058, 9058, 9058], dtype=int64))

In [21]:
# Fit LogReg model
log_reg = LogisticRegression()
log_reg.fit(X_train_samp,y_train_samp)

c:\users\spenc\anaconda3\envs\cv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [22]:
y_hat = log_reg.predict(X_test)
print(f'LogReg accuracy on held-out frames = {round(accuracy_score(y_test, y_hat),4)}')

LogReg accuracy on held-out frames = 0.9025


In [23]:
confusion_matrix(y_test, y_hat, labels=[-1, 38, 40])
target_names = ['nothing', 'up', 'down']
print(classification_report(y_test, y_hat, target_names=target_names))

              precision    recall  f1-score   support

     nothing       0.94      0.94      0.94      3051
          up       0.53      0.53      0.53       361
        down       0.96      0.95      0.96       507

    accuracy                           0.90      3919
   macro avg       0.81      0.81      0.81      3919
weighted avg       0.90      0.90      0.90      3919



In [24]:
pickle.dump(log_reg, open('Existing_Models/log-reg.pkl', 'wb'))